In [176]:
# Python
import pandas as pd
import numpy as np
from collections import OrderedDict
#
import importlib
import utilities.variables as variables
import utilities.utility as utility
import utilities.plots as plots
import utilities.prophet_utils as prophet_utils
import utilities.mpt_utils as mpt_utils
importlib.reload(variables)
importlib.reload(utility)
importlib.reload(plots)
importlib.reload(prophet_utils)
#
import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

In [177]:
# Includes by default percentage values
df = pd.read_csv('../../../data/df_monthly_prices_complete_euro.csv', index_col='Date')
df_pct = pd.read_csv('../../../data/df_monthly_returns_complete.csv', index_col='Date')
df_overview = pd.read_csv('../../../data/df_overview.csv', index_col=0)

In [178]:
df

,RS1.L,KE,TEG.DE,LEG.DE,SCS,HNI,AVT,ACCO,VNA.DE,7912.T,...,DEQ.DE,KIDS,HALO,MATW,9842.T,KVHI,MOON.L,NEO,6055.T,UNP
Date,,,,,,,,,,,,,,,,,,,,,
1999-10-01,225.137182,10.181600,44.819553,30.297156,4.545932,8.601178,18.664071,10.674827,8.585901,12.452896,...,7.348418,20.900000,2.581920,8.076970,2.402565,2.6950,228.688201,0.35200,0.349532,7.558170
1999-11-01,255.238082,10.181600,44.819553,30.297156,4.680961,9.477732,18.792348,11.015312,8.585901,12.810578,...,7.348418,20.900000,2.581920,7.488379,2.402565,2.6675,228.688201,0.35200,0.349532,6.380382
1999-12-01,282.658707,10.181600,44.819553,30.297156,4.320888,9.657366,20.695110,11.015312,8.585901,12.482466,...,7.348418,20.900000,2.581920,8.857225,2.402565,2.6950,228.688201,0.35200,0.349532,5.922826
2000-01-01,288.505511,10.181600,44.819553,30.297156,4.136283,8.474268,18.412752,8.037073,8.585901,12.482466,...,7.348418,20.900000,2.581920,7.488379,2.402565,3.3000,228.688201,0.35200,0.349532,5.480240
2000-02-01,288.505511,10.331200,36.391943,37.196711,4.454461,8.116590,22.935556,15.700016,10.089434,11.549095,...,7.534067,19.611680,2.766720,7.805783,3.542987,6.4900,258.219000,0.33616,0.353100,5.173892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,807.658412,20.169600,14.260000,78.867676,11.929318,40.839199,47.770988,4.347898,27.895920,30.280800,...,18.555727,27.790400,38.975201,24.487249,11.763362,4.5584,187.200000,12.06480,12.052800,202.641619
2024-06-01,804.788256,19.342400,13.670000,76.260002,11.318006,39.366952,45.049968,4.074207,26.549999,33.554400,...,19.906872,25.308800,46.076801,21.845480,11.916610,4.0920,223.002007,12.20560,13.726800,198.055067
2024-07-01,957.060000,20.856001,13.950000,80.860001,12.654159,48.050067,47.308799,4.429617,28.410000,30.814000,...,21.843512,27.086401,48.628799,25.246574,11.156497,3.9160,255.060000,15.60240,12.387600,215.973331


### Train & Plot Methods

In [179]:
#filtered_cols = [col for col in df.columns if not '.T' in col]
#df = df[filtered_cols][600:800]
df_pct.drop(columns=['2181.T', 'BGEO.L', 'ILM1.DE', 'MCG.L', '8766.T'], inplace=True)

In [180]:
test_size_years = variables.FIVE_YEARS_NR
# actual values
df_train = df_pct.head(int(variables.ALL_YEARS_NR - test_size_years) * 12)
df_test = df_pct.tail(test_size_years * 12)

### 1 Month Actual vs Prediction

In [181]:
importlib.reload(prophet_utils)
importlib.reload(plots)

forecasts_1m = prophet_utils.train_predict(dataframe=df_train, months=1)
# 
plots.plot_lines_actual_vs_predicted(df_pct=df_train, forecasts=forecasts_1m, months=1)

Optimization terminated abnormally. Falling back to Newton.


#### Allocation - 1 Month

In [182]:
importlib.reload(prophet_utils)
importlib.reload(mpt_utils)

df_forecast_1m = prophet_utils.get_df_from_forecast(forecasts_1m)
df_forecast_1m = df_forecast_1m - 1
df_forecast_1m  # (0.02 for 2%)

weights_1m, mu_1m, S_1m, weights_all_1m = mpt_utils.portfolio_and_plot(df_forecast_1m, df)

Expected annual return: 39.7%
Annual volatility: 3.1%
Sharpe Ratio: 12.09
-- Allocation --
{'STAA': 1, 'SLP': 1, 'BIRD': 2, '7846.T': 1, 'ATRI': 1, 'VAR1.DE': 1, 'ARLO': 78, '9068.T': 3, 'WAF.DE': 1, 'CPRX': 1, 'MRNA': 4, 'CLVT': 5, 'FTK.DE': 3, 'QSI': 59, 'DLAR.L': 1, '7947.T': 1, '7844.T': 10, 'BLD': 1, '7419.T': 3, '8086.T': 8, 'ACCO': 10, 'GIII': 1, 'OSIS': 1, 'MRTN': 4, 'PSDL.L': 1, '2395.T': 7, 'HCSG': 3, 'ADUS': 1, 'PEN': 1, 'LITE': 1, '7994.T': 7, 'FUBO': 6, '9413.T': 3, '2733.T': 2, 'IOT': 1, '5105.T': 3, 'MPX': 4, 'SONO': 4, '2792.T': 10, '7780.T': 1, 'TWKS': 9, '5938.T': 3, '8173.T': 3, '9069.T': 6, '6287.T': 1, 'CMLS': 3, '9882.T': 1, '7976.T': 3, 'CENT': 1, 'TLYS': 4, 'PSTG': 1, '6651.T': 1, 'SCHL': 1, 'EQC': 1, 'DAKT': 10, '6460.T': 3, 'RDFN': 2, 'B4B.DE': 3, 'ZM': 1, '4544.T': 3, 'OMI': 10, 'INS.DE': 5, 'U': 1, 'ALGM': 1, '3668.T': 1, '2331.T': 1, 'ESCA': 2, 'CALX': 2, '6841.T': 1, 'CERT': 2, 'HPE': 1, 'BELFB': 1, '3086.T': 2, 'KD': 1, 'ALIT': 3, 'SPWH': 2, 'EVLV': 6, 'D

/Users/herbishtini/anaconda3/envs/portfolio_optimisation/lib/python3.11/site-packages/pypfopt/discrete_allocation.py:27: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



#### Overview table - 1 Month

In [183]:
importlib.reload(mpt_utils)
# Create overview
mpt_utils.generate_overview_table(weights_all_1m, mu_1m, S_1m, df_pct)

,Weight,Average Covariance,Average Returns,Return Last 12 Months,Return (Actual) Next 12 Months
STAA,1,0.022065,23.30%,-17.50%,118.28%
SLP,1,-0.007065,20.20%,77.26%,118.41%
BIRD,2,0.006196,21.48%,11.18%,123.44%
7846.T,1,0.005652,21.46%,-29.30%,-20.84%
ATRI,1,0.010435,-4.31%,22.70%,-18.43%
...,...,...,...,...,...
CRSR,1,0.004130,10.94%,-42.70%,93.37%
EBOX.L,12,0.000652,-16.12%,-3.31%,-2.38%
SKT,1,0.004130,66.12%,-22.60%,-57.51%
NMRK,1,0.000652,52.91%,14.06%,-50.44%


### 6 months Actual vs Prediction

In [184]:
importlib.reload(prophet_utils)
importlib.reload(plots)

forecasts_6m = prophet_utils.train_predict(dataframe=df_train, months=6)
#
plots.plot_lines_actual_vs_predicted(df_pct=df_train, forecasts=forecasts_6m, months=6)


Optimization terminated abnormally. Falling back to Newton.


#### Allocation

In [185]:
importlib.reload(prophet_utils)

df_forecast_6m = prophet_utils.get_df_from_forecast(forecasts_6m)
df_forecast_6m = df_forecast_6m - 1
df_forecast_6m # (0.02 for 2%)

,1435.T,1911.T,1925.T,1928.T,1973.T,2120.T,2124.T,2146.T,2154.T,2168.T,...,ZD,ZI,ZIG.L,ZIL2.DE,ZIM,ZM,ZS,ZTS,ZUMZ,ZWS
ds,,,,,,,,,,,,,,,,,,,,,
1999-11-01,0.013764,0.006057,-0.024695,-0.002811,-0.001982,0.146841,0.032647,-0.148546,-0.050828,0.000043,...,0.035414,-0.007713,-0.037991,-0.001012,0.011180,-0.015694,0.018186,0.028212,-0.024039,0.008309
1999-12-01,0.013635,-0.014938,0.023676,-0.019568,0.015492,0.063795,-0.026163,0.005641,-0.013059,-0.041680,...,0.056778,0.009157,0.033899,0.029595,-0.018249,0.018495,0.067873,-0.013469,-0.021230,-0.022484
2000-01-01,-0.012925,-0.012619,-0.002361,-0.009497,-0.014455,-0.043867,0.099272,-0.013524,-0.030936,-0.015538,...,-0.061091,0.001950,-0.031633,0.027094,0.002209,0.011084,-0.034820,-0.016982,-0.011984,0.006259
2000-02-01,0.031938,0.026228,-0.010279,0.007604,-0.006429,0.099813,1.130689,0.100704,0.317025,0.035686,...,0.064623,-0.015806,0.021494,0.005884,0.018648,0.008067,-0.033727,0.008614,0.068131,0.032422
2000-03-01,0.061253,0.029970,0.041845,0.043666,0.042885,0.076425,0.799437,-0.047407,0.201190,0.046915,...,0.015466,-0.004253,-0.068869,-0.032572,0.006608,0.017444,0.011670,0.001947,0.057701,0.031024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-30,0.005996,-0.098081,-0.000114,-0.082020,0.054705,0.018843,-0.612155,0.074447,-0.173054,-0.054946,...,0.095690,-0.012509,0.060610,-0.029562,-0.068225,0.061970,0.184027,0.004959,-0.010487,-0.013638
2019-12-31,-0.033229,0.010034,0.024037,-0.000974,0.057079,-0.175851,0.063518,0.037313,-0.089934,0.048096,...,-0.535412,-0.029121,-0.189493,-0.000211,0.006520,0.027087,-0.040343,0.000435,0.001539,0.019997
2020-01-31,0.023380,0.125234,0.035050,0.081407,0.066726,0.294302,2.539711,0.276429,0.874066,0.145274,...,0.041519,-0.074560,0.078067,-0.039364,0.028056,-0.058998,-0.114048,0.050892,0.080143,0.085356


In [186]:
importlib.reload(mpt_utils)

weights_6m, mu_6m, S_6m, weights_all_6m = mpt_utils.portfolio_and_plot(df_forecast_6m, df)

Expected annual return: 55.3%
Annual volatility: 6.5%
Sharpe Ratio: 8.24
-- Allocation --
{'STAA': 1, 'SLP': 1, 'CPRX': 2, 'VAR1.DE': 1, 'ATRI': 1, 'BIRD': 1, '7846.T': 1, 'FTK.DE': 4, '9068.T': 3, 'ARLO': 81, 'BLD': 1, '7947.T': 1, 'ADUS': 1, '8086.T': 9, 'CLVT': 4, '7419.T': 3, 'PEN': 1, 'MRTN': 3, 'ACCO': 11, '9413.T': 3, '5105.T': 4, '7844.T': 10, 'GIII': 1, '7575.T': 1, 'QSI': 50, '2733.T': 2, 'PSDL.L': 1, 'TPL': 1, '7780.T': 2, '8173.T': 3, 'DLAR.L': 1, 'HCSG': 3, 'SONO': 4, '7994.T': 7, 'FUBO': 5, '5938.T': 3, 'IOT': 1, 'MRNA': 2, 'EQC': 2, 'MPX': 4, 'RUN': 1, '9069.T': 6, '2395.T': 5, 'CMLS': 3, '2792.T': 5, '9882.T': 1, 'TWKS': 7, '6287.T': 1, '6651.T': 1, 'SCHL': 1, 'RDFN': 2, '9869.T': 1, 'B4B.DE': 3, 'PSTG': 1, 'TLYS': 3, '4544.T': 3, '3668.T': 1, 'DAKT': 9, '7976.T': 2, 'CENT': 1, 'INS.DE': 4, 'CALX': 2, 'CIO': 2, 'HPE': 1, '1973.T': 1, 'HABA.DE': 2, 'UPBD': 1, 'ALGM': 1, '6841.T': 1, '2331.T': 1, '3086.T': 1, '9697.T': 1, '6460.T': 1, 'SPWH': 1, 'THRY': 1, '3421.T': 1, 'B

/Users/herbishtini/anaconda3/envs/portfolio_optimisation/lib/python3.11/site-packages/pypfopt/discrete_allocation.py:27: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



#### Overview

In [187]:
importlib.reload(mpt_utils)
# Create overview
mpt_utils.generate_overview_table(weights_all_6m, mu_6m, S_6m, df_pct)

,Weight,Average Covariance,Average Returns,Return Last 12 Months,Return (Actual) Next 12 Months
STAA,1,0.014146,26.47%,-17.50%,118.28%
SLP,1,-0.006951,21.25%,77.26%,118.41%
CPRX,2,0.000976,21.36%,58.04%,-44.43%
VAR1.DE,1,-0.016951,23.91%,261.53%,32.91%
ATRI,1,0.013537,5.23%,22.70%,-18.43%
...,...,...,...,...,...
ONL,2,0.001585,8.14%,11.28%,24.36%
3402.T,2,0.013537,45.81%,-0.46%,-39.31%
OMI,3,0.005976,11.09%,-13.53%,325.77%
APLD,1,0.006585,4.65%,-6.98%,3.37%


### 12 months Actual vs Prediction

In [188]:
importlib.reload(prophet_utils)
importlib.reload(plots)

forecasts_12m = prophet_utils.train_predict(dataframe=df_train, months=12)
#
plots.plot_lines_actual_vs_predicted(df_pct=df_train, forecasts=forecasts_12m, months=12)

Optimization terminated abnormally. Falling back to Newton.


#### Allocation - 12M

Generate return dataframe from Prophet dictionary

In [189]:
importlib.reload(prophet_utils)

df_forecast_12m = prophet_utils.get_df_from_forecast(forecasts_12m)
df_forecast_12m = df_forecast_12m - 1
df_forecast_12m # (0.02 for 2%)

,1435.T,1911.T,1925.T,1928.T,1973.T,2120.T,2124.T,2146.T,2154.T,2168.T,...,ZD,ZI,ZIG.L,ZIL2.DE,ZIM,ZM,ZS,ZTS,ZUMZ,ZWS
ds,,,,,,,,,,,,,,,,,,,,,
1999-11-01,0.013764,0.006057,-0.024695,-0.002811,-0.001982,0.146841,0.032647,-0.148546,-0.050828,0.000043,...,0.035414,-0.007713,-0.037991,-0.001012,0.011180,-0.015694,0.018186,0.028212,-0.024039,0.008309
1999-12-01,0.013635,-0.014938,0.023676,-0.019568,0.015492,0.063795,-0.026163,0.005641,-0.013059,-0.041680,...,0.056778,0.009157,0.033899,0.029595,-0.018249,0.018495,0.067873,-0.013469,-0.021230,-0.022484
2000-01-01,-0.012925,-0.012619,-0.002361,-0.009497,-0.014455,-0.043867,0.099272,-0.013524,-0.030936,-0.015538,...,-0.061091,0.001950,-0.031633,0.027094,0.002209,0.011084,-0.034820,-0.016982,-0.011984,0.006259
2000-02-01,0.031938,0.026228,-0.010279,0.007604,-0.006429,0.099813,1.130689,0.100704,0.317025,0.035686,...,0.064623,-0.015806,0.021494,0.005884,0.018648,0.008067,-0.033727,0.008614,0.068131,0.032422
2000-03-01,0.061253,0.029970,0.041845,0.043666,0.042885,0.076425,0.799437,-0.047407,0.201190,0.046915,...,0.015466,-0.004253,-0.068869,-0.032572,0.006608,0.017444,0.011670,0.001947,0.057701,0.031024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-31,0.008891,0.007171,-0.000652,-0.013101,0.043683,-0.052541,-0.009970,0.104380,-0.030915,0.107507,...,0.004482,-0.070377,0.015969,-0.067569,-0.006298,-0.012174,0.096031,0.024892,-0.005110,0.032418
2020-06-30,0.009727,-0.022059,0.018628,-0.003588,0.008333,-0.007333,0.059206,0.101516,0.068215,0.055834,...,-0.126941,-0.053532,-0.047652,-0.083720,-0.005765,-0.020719,0.001417,0.017857,0.016062,-0.019308
2020-07-31,-0.040119,-0.027426,-0.043254,0.013963,-0.013363,0.014471,-0.147351,-0.002936,0.190807,-0.031933,...,0.007897,-0.029255,-0.033055,-0.051241,-0.021386,-0.033229,0.008989,0.039298,0.009471,-0.015998


In [190]:
importlib.reload(mpt_utils)
importlib.reload(mpt_utils)

weights_12m, mu_12m, S_12m, weights_all_12m = mpt_utils.portfolio_and_plot(df_forecast_12m, df)

Expected annual return: 63.7%
Annual volatility: 6.8%
Sharpe Ratio: 9.11
-- Allocation --
{'STAA': 1, 'SLP': 1, 'CPRX': 2, 'VAR1.DE': 1, '7846.T': 1, 'FTK.DE': 4, '9068.T': 3, 'ACCO': 18, 'ATRI': 1, 'BIRD': 1, 'ARLO': 91, 'MRTN': 4, 'BLD': 1, '8086.T': 10, '9413.T': 4, 'ADUS': 1, '7419.T': 3, 'PEN': 1, '7947.T': 1, 'DLAR.L': 2, 'CLVT': 4, '8173.T': 4, '5105.T': 4, '7844.T': 10, 'GIII': 1, 'QSI': 59, '7780.T': 2, '7575.T': 1, 'TPL': 1, 'SONO': 5, '2733.T': 2, '5938.T': 4, 'PRG': 1, 'RUN': 1, 'EQC': 3, '7994.T': 8, 'HCSG': 3, 'FUBO': 6, '6383.T': 1, '9069.T': 6, 'OMCL': 1, 'CMLS': 4, '2792.T': 6, 'IOT': 1, '2395.T': 5, 'MRNA': 2, 'PAGS': 1, 'MPX': 3, '9882.T': 1, '6651.T': 1, '9869.T': 1, 'TWKS': 6, 'B4B.DE': 3, '6200.T': 1, 'RDFN': 2, '4544.T': 4, 'SCHL': 1, '1973.T': 1, '7976.T': 3, 'CALX': 3, 'UPBD': 1, 'PSTG': 1, 'CIO': 2, 'DAKT': 10, 'INS.DE': 5, 'HABA.DE': 2, 'TLYS': 2, 'HPE': 1, '5970.T': 1, 'SCS': 1, '3421.T': 2, 'BIG': 1, '3402.T': 3, 'THRY': 2, '3086.T': 2, 'CENT': 1, '6460.T':

/Users/herbishtini/anaconda3/envs/portfolio_optimisation/lib/python3.11/site-packages/pypfopt/discrete_allocation.py:27: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



#### Overview - 12 Months

In [191]:
importlib.reload(mpt_utils)
# Create overview
mpt_utils.generate_overview_table(weights_all_12m, mu_12m, S_12m, df_pct)

,Weight,Average Covariance,Average Returns,Return Last 12 Months,Return (Actual) Next 12 Months
STAA,1,0.011744,27.55%,-17.50%,118.28%
SLP,1,-0.008140,20.95%,77.26%,118.41%
CPRX,2,0.003837,24.94%,58.04%,-44.43%
VAR1.DE,1,-0.020930,5.57%,261.53%,32.91%
7846.T,1,-0.007558,5.08%,-29.30%,-20.84%
...,...,...,...,...,...
OMI,4,0.000814,7.97%,-13.53%,325.77%
EBOX.L,13,0.000233,46.89%,-3.31%,-2.38%
PTVE,1,0.000465,11.17%,-12.72%,12.89%
RNW,1,0.003256,4.65%,-0.13%,1.61%
